In [39]:
# read in ../parquets_version3/votes.parquet
import pyarrow.parquet as pq

votes = pq.read_table('../parquets_version3/votes.parquet')
votes

pyarrow.Table
platform: string
platform_deployment_id: string
proposal_id: string
vote_id: string
voter: string
date: timestamp[us]
choice: string
weight: decimal128(38, 4)
----
platform: [["realms","realms","realms","realms","realms",...,"tally","tally","tally","tally","tally"],["tally","tally","tally","tally","tally",...,"tally","tally","tally","tally","tally"],...,["snapshot","snapshot","snapshot","snapshot","snapshot",...,"snapshot","snapshot","snapshot","snapshot","snapshot"],["snapshot","snapshot","snapshot","snapshot","snapshot",...,"snapshot","snapshot","snapshot","snapshot","snapshot"]]
platform_deployment_id: [["84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ",...,"eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10","eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10","eip155:10:0xcDF27F107

In [40]:
# 1. make a deployment_id column from <platform>_<platform_deployment_id>
# 2. make an id column from <platform>_<platform_deployment_id>_<proposal_id>_<vote_id>
# 3. rename vote_id to platform_vote_id
# 4. columns should be: id, proposal_id (uuid), deployment_id (uuid), platform_vote_id, voter, date, choice, weight

In [41]:
# keep only 100000 votes
# FIXME: remove this!
# votes = votes[:100000]

### 1. make a deployment_id column from <platform>_<platform_deployment_id>

In [42]:
deployment_ids_and_platform_deployment_ids = votes.select(['platform', 'platform_deployment_id'])
deployment_ids_and_platform_deployment_ids_df = deployment_ids_and_platform_deployment_ids.to_pandas()
deployment_ids_and_platform_deployment_ids_df

,platform,platform_deployment_id
0,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ
1,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ
2,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ
3,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ
4,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ
...,...,...
21617895,snapshot,linea-build.eth
21617896,snapshot,linea-build.eth
21617897,snapshot,linea-build.eth
21617898,snapshot,linea-build.eth


In [43]:
# make a to_hash column from platform and platform_deployment_id
deployment_ids_and_platform_deployment_ids_df['to_hash'] = deployment_ids_and_platform_deployment_ids_df['platform'] + '_' + deployment_ids_and_platform_deployment_ids_df['platform_deployment_id']
deployment_ids_and_platform_deployment_ids_df

,platform,platform_deployment_id,to_hash
0,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
1,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
2,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
3,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
4,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
...,...,...,...
21617895,snapshot,linea-build.eth,snapshot_linea-build.eth
21617896,snapshot,linea-build.eth,snapshot_linea-build.eth
21617897,snapshot,linea-build.eth,snapshot_linea-build.eth
21617898,snapshot,linea-build.eth,snapshot_linea-build.eth


In [44]:
# make a deployment_id column from to_hash
import uuid
deployment_ids_and_platform_deployment_ids_df['deployment_id'] = deployment_ids_and_platform_deployment_ids_df['to_hash'].apply(lambda x: str(uuid.uuid5(uuid.NAMESPACE_OID, str(x))))
deployment_ids_and_platform_deployment_ids_df

,platform,platform_deployment_id,to_hash,deployment_id
0,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,f3ef168e-d216-573c-ba95-ff15d29069fe
1,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,f3ef168e-d216-573c-ba95-ff15d29069fe
2,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,f3ef168e-d216-573c-ba95-ff15d29069fe
3,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,f3ef168e-d216-573c-ba95-ff15d29069fe
4,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,f3ef168e-d216-573c-ba95-ff15d29069fe
...,...,...,...,...
21617895,snapshot,linea-build.eth,snapshot_linea-build.eth,eeed4e65-5607-59e2-a003-fddc8db9ff09
21617896,snapshot,linea-build.eth,snapshot_linea-build.eth,eeed4e65-5607-59e2-a003-fddc8db9ff09
21617897,snapshot,linea-build.eth,snapshot_linea-build.eth,eeed4e65-5607-59e2-a003-fddc8db9ff09
21617898,snapshot,linea-build.eth,snapshot_linea-build.eth,eeed4e65-5607-59e2-a003-fddc8db9ff09


In [45]:
# take the deployment_id column and make it a pyarrow array
deployment_ids = deployment_ids_and_platform_deployment_ids_df['deployment_id'].to_numpy()
deployment_ids

array(['f3ef168e-d216-573c-ba95-ff15d29069fe',
       'f3ef168e-d216-573c-ba95-ff15d29069fe',
       'f3ef168e-d216-573c-ba95-ff15d29069fe', ...,
       'eeed4e65-5607-59e2-a003-fddc8db9ff09',
       'eeed4e65-5607-59e2-a003-fddc8db9ff09',
       'eeed4e65-5607-59e2-a003-fddc8db9ff09'], dtype=object)

In [46]:
import pyarrow as pa
# add deployment_id column to votes table
votes = votes.append_column('deployment_id', pa.array(deployment_ids))
votes

pyarrow.Table
platform: string
platform_deployment_id: string
proposal_id: string
vote_id: string
voter: string
date: timestamp[us]
choice: string
weight: decimal128(38, 4)
deployment_id: string
----
platform: [["realms","realms","realms","realms","realms",...,"tally","tally","tally","tally","tally"],["tally","tally","tally","tally","tally",...,"tally","tally","tally","tally","tally"],...,["snapshot","snapshot","snapshot","snapshot","snapshot",...,"snapshot","snapshot","snapshot","snapshot","snapshot"],["snapshot","snapshot","snapshot","snapshot","snapshot",...,"snapshot","snapshot","snapshot","snapshot","snapshot"]]
platform_deployment_id: [["84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ",...,"eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10","eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10",

### 2. make an id column from \<platform>\_<platform_deployment_id>\_<proposal_id>\_<vote_id>


In [47]:
ids_df = votes.select(['platform', 'platform_deployment_id', 'proposal_id', 'vote_id']).to_pandas()
ids_df


,platform,platform_deployment_id,proposal_id,vote_id
0,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,HFhP5bFDBq6sP54RrkxCir75793686LFCCqbrFRAzgAM
1,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,7Cw6ukhy4wuUnFwgqDAA7m7VkVV5QWiGtDRwVmadx5aZ
2,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,91z2tkWCnEBo7SyoHogqnnRQURQgi1LEKevwwpa5s3UV
3,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,C2E43Xunhb4xPwcqrb5fvg9MXktJbvUNHGdLiXNKHrci
4,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,AvywQhmbye96npitAKK41ETVWHLqXvDarGpVqGfEcTMG
...,...,...,...,...
21617895,snapshot,linea-build.eth,0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96...,0xacc2622f74ec8a34e47eb55600dbb6d85b4d013a3a97...
21617896,snapshot,linea-build.eth,0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96...,0xc32a85762ea21a7659c010c85eebaef0d157b58eb3cc...
21617897,snapshot,linea-build.eth,0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96...,0xd5c7c04fea074118091e02dc824317457f8195515e73...
21617898,snapshot,linea-build.eth,0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96...,0xec43ff56a98e70c31b809f238e34593c8744a34610a1...


In [48]:
# make a to_hash column from platform, platform_deployment_id, proposal_id, and vote_id
ids_df['to_hash'] = ids_df['platform'] + '_' + ids_df['platform_deployment_id'] + '_' + ids_df['proposal_id'] + '_' + ids_df['vote_id']
ids_df

,platform,platform_deployment_id,proposal_id,vote_id,to_hash
0,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,HFhP5bFDBq6sP54RrkxCir75793686LFCCqbrFRAzgAM,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
1,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,7Cw6ukhy4wuUnFwgqDAA7m7VkVV5QWiGtDRwVmadx5aZ,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
2,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,91z2tkWCnEBo7SyoHogqnnRQURQgi1LEKevwwpa5s3UV,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
3,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,C2E43Xunhb4xPwcqrb5fvg9MXktJbvUNHGdLiXNKHrci,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
4,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,AvywQhmbye96npitAKK41ETVWHLqXvDarGpVqGfEcTMG,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
...,...,...,...,...,...
21617895,snapshot,linea-build.eth,0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96...,0xacc2622f74ec8a34e47eb55600dbb6d85b4d013a3a97...,snapshot_linea-build.eth_0xda4f201a37ea08cf189...
21617896,snapshot,linea-build.eth,0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96...,0xc32a85762ea21a7659c010c85eebaef0d157b58eb3cc...,snapshot_linea-build.eth_0xda4f201a37ea08cf189...
21617897,snapshot,linea-build.eth,0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96...,0xd5c7c04fea074118091e02dc824317457f8195515e73...,snapshot_linea-build.eth_0xda4f201a37ea08cf189...
21617898,snapshot,linea-build.eth,0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96...,0xec43ff56a98e70c31b809f238e34593c8744a34610a1...,snapshot_linea-build.eth_0xda4f201a37ea08cf189...


In [49]:
ids_df = ids_df[['to_hash']]
ids_df

,to_hash
0,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
1,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
2,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
3,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
4,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
...,...
21617895,snapshot_linea-build.eth_0xda4f201a37ea08cf189...
21617896,snapshot_linea-build.eth_0xda4f201a37ea08cf189...
21617897,snapshot_linea-build.eth_0xda4f201a37ea08cf189...
21617898,snapshot_linea-build.eth_0xda4f201a37ea08cf189...


In [50]:
ids_df.iloc[9]['to_hash']

'realms_9mr8UCn3DTP7JbUvRsnJSnMb6TSnsZNdPFUE2j652GXv_FEfR77XcgXCgqouuJbhC9crhRbSEPcyKpyYpw8qMsQQX_J76bcUp2KNzUCp9uLYuDUtVp71mSJPycTP39bGF9bb2v'

In [51]:
# hash the to_hash column
ids_df['id'] = ids_df['to_hash'].apply(lambda x: str(uuid.uuid5(uuid.NAMESPACE_OID, str(x))))
ids_df

/var/folders/yn/b6240_995k3gl4vwtnbzzy480000gq/T/ipykernel_43171/1175088504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ids_df['id'] = ids_df['to_hash'].apply(lambda x: str(uuid.uuid5(uuid.NAMESPACE_OID, str(x))))


,to_hash,id
0,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,6755eb11-f57b-5841-961b-ededa4411a91
1,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,0c79521b-223f-5174-8e00-e7b96bc1b871
2,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,e9895866-0f92-5fd7-945f-3c04dd3692c1
3,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,e1aada30-aa76-57f1-85d3-9c06da614245
4,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,2de4240a-700a-5621-a09b-b57057538f2b
...,...,...
21617895,snapshot_linea-build.eth_0xda4f201a37ea08cf189...,ead97bb6-53d5-51f9-926e-936dd4ea6389
21617896,snapshot_linea-build.eth_0xda4f201a37ea08cf189...,b3057b00-dce3-579e-bfc3-34c7a9a2fd0b
21617897,snapshot_linea-build.eth_0xda4f201a37ea08cf189...,aeb3edbf-5d9d-5e9e-b2fc-c04eb96323c9
21617898,snapshot_linea-build.eth_0xda4f201a37ea08cf189...,9812961d-af6b-58b1-aff0-6dd455276d8b


In [52]:
# make a numpy array from the id column
ids = ids_df['id'].to_numpy()
ids

array(['6755eb11-f57b-5841-961b-ededa4411a91',
       '0c79521b-223f-5174-8e00-e7b96bc1b871',
       'e9895866-0f92-5fd7-945f-3c04dd3692c1', ...,
       'aeb3edbf-5d9d-5e9e-b2fc-c04eb96323c9',
       '9812961d-af6b-58b1-aff0-6dd455276d8b',
       '54ddf534-498f-5832-9974-44ed4a57ef8a'], dtype=object)

In [53]:
# set the id column to the ids array
votes = votes.append_column('id', pa.array(ids))
votes

pyarrow.Table
platform: string
platform_deployment_id: string
proposal_id: string
vote_id: string
voter: string
date: timestamp[us]
choice: string
weight: decimal128(38, 4)
deployment_id: string
id: string
----
platform: [["realms","realms","realms","realms","realms",...,"tally","tally","tally","tally","tally"],["tally","tally","tally","tally","tally",...,"tally","tally","tally","tally","tally"],...,["snapshot","snapshot","snapshot","snapshot","snapshot",...,"snapshot","snapshot","snapshot","snapshot","snapshot"],["snapshot","snapshot","snapshot","snapshot","snapshot",...,"snapshot","snapshot","snapshot","snapshot","snapshot"]]
platform_deployment_id: [["84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ",...,"eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10","eip155:10:0xcDF27F107725988f2261Ce2256bDfCd

### 2.1 create proposal id column like <platform>\_<platform_deployment_id>\_<proposal_id>

In [54]:
# row['platform'] + '_' + str(row['platform_deployment_id']) + '_' + str(row['proposal_id']))
proposal_ids_df = votes.select(['platform', 'platform_deployment_id', 'proposal_id']).to_pandas()
proposal_ids_df['to_hash'] = proposal_ids_df['platform'] + '_' + proposal_ids_df['platform_deployment_id'] + '_' + proposal_ids_df['proposal_id']
proposal_ids_df.head()

,platform,platform_deployment_id,proposal_id,to_hash
0,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
1,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
2,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
3,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...
4,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...


In [55]:
# hash the to_hash column
proposal_ids_df['hashed_proposal_id'] = proposal_ids_df['to_hash'].apply(lambda x: str(uuid.uuid5(uuid.NAMESPACE_OID, str(x))))
proposal_ids_df.head()

,platform,platform_deployment_id,proposal_id,to_hash,hashed_proposal_id
0,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,a7f83165-6b46-5aa5-a363-8c803825e9db
1,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,a7f83165-6b46-5aa5-a363-8c803825e9db
2,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,a7f83165-6b46-5aa5-a363-8c803825e9db
3,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,a7f83165-6b46-5aa5-a363-8c803825e9db
4,realms,84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ,Bc7C6bycdbTSpd21NtVqHo2jn3N7GCMYz8PZz3pz6Kob,realms_84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm3...,a7f83165-6b46-5aa5-a363-8c803825e9db


In [56]:
# numpy array of hashed_proposal_id
hashed_proposal_ids = proposal_ids_df['hashed_proposal_id'].to_numpy()
hashed_proposal_ids

array(['a7f83165-6b46-5aa5-a363-8c803825e9db',
       'a7f83165-6b46-5aa5-a363-8c803825e9db',
       'a7f83165-6b46-5aa5-a363-8c803825e9db', ...,
       'e7f552e3-64a9-5efc-a5c5-ddf124ef1e01',
       'e7f552e3-64a9-5efc-a5c5-ddf124ef1e01',
       'e7f552e3-64a9-5efc-a5c5-ddf124ef1e01'], dtype=object)

In [57]:
votes = votes.append_column('proposal_id_uuid', pa.array(hashed_proposal_ids))
votes

pyarrow.Table
platform: string
platform_deployment_id: string
proposal_id: string
vote_id: string
voter: string
date: timestamp[us]
choice: string
weight: decimal128(38, 4)
deployment_id: string
id: string
proposal_id_uuid: string
----
platform: [["realms","realms","realms","realms","realms",...,"tally","tally","tally","tally","tally"],["tally","tally","tally","tally","tally",...,"tally","tally","tally","tally","tally"],...,["snapshot","snapshot","snapshot","snapshot","snapshot",...,"snapshot","snapshot","snapshot","snapshot","snapshot"],["snapshot","snapshot","snapshot","snapshot","snapshot",...,"snapshot","snapshot","snapshot","snapshot","snapshot"]]
platform_deployment_id: [["84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ","84pGFuy1Y27ApK67ApethaPvexeDWA66zNV8gm38TVeQ",...,"eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10","eip155:10:0xcDF27F

### 3. and 4. rename vote_id to platform_vote_id and update columns

columns should be: id, proposal_id (uuid), deployment_id (uuid), platform_vote_id, voter, date, choice, weight

In [58]:
votes.column_names

['platform',
 'platform_deployment_id',
 'proposal_id',
 'vote_id',
 'voter',
 'date',
 'choice',
 'weight',
 'deployment_id',
 'id',
 'proposal_id_uuid']

In [59]:
votes = votes.select([
    'id',
    'proposal_id_uuid', # to be renamed to proposal_id
    'deployment_id',
    'vote_id', # to be renamed to platform_vote_id
    'voter',
    'date',
    'choice',
    'weight'
])
votes.column_names

['id',
 'proposal_id_uuid',
 'deployment_id',
 'vote_id',
 'voter',
 'date',
 'choice',
 'weight']

In [60]:
votes = votes.rename_columns([
 'id',
 'proposal_id',
 'deployment_id',
 'platform_vote_id',
 'voter',
 'date',
 'choice',
 'weight'])
votes

pyarrow.Table
id: string
proposal_id: string
deployment_id: string
platform_vote_id: string
voter: string
date: timestamp[us]
choice: string
weight: decimal128(38, 4)
----
id: [["6755eb11-f57b-5841-961b-ededa4411a91","0c79521b-223f-5174-8e00-e7b96bc1b871","e9895866-0f92-5fd7-945f-3c04dd3692c1","e1aada30-aa76-57f1-85d3-9c06da614245","2de4240a-700a-5621-a09b-b57057538f2b",...,"ead97bb6-53d5-51f9-926e-936dd4ea6389","b3057b00-dce3-579e-bfc3-34c7a9a2fd0b","aeb3edbf-5d9d-5e9e-b2fc-c04eb96323c9","9812961d-af6b-58b1-aff0-6dd455276d8b","54ddf534-498f-5832-9974-44ed4a57ef8a"]]
proposal_id: [["a7f83165-6b46-5aa5-a363-8c803825e9db","a7f83165-6b46-5aa5-a363-8c803825e9db","a7f83165-6b46-5aa5-a363-8c803825e9db","a7f83165-6b46-5aa5-a363-8c803825e9db","a7f83165-6b46-5aa5-a363-8c803825e9db",...,"e7f552e3-64a9-5efc-a5c5-ddf124ef1e01","e7f552e3-64a9-5efc-a5c5-ddf124ef1e01","e7f552e3-64a9-5efc-a5c5-ddf124ef1e01","e7f552e3-64a9-5efc-a5c5-ddf124ef1e01","e7f552e3-64a9-5efc-a5c5-ddf124ef1e01"]]
deployment_id: 

In [61]:
# save to parquet
pq.write_table(votes, '../parquets_version3/votes_new.parquet')